# Metadata

In [ ]:
import os
import json
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import soundfile as sf
from IPython.display import display, Audio

### Dataset

**Standard Placement**

In [ ]:
DATA_DIR = '/Users/richanamballa/Desktop/binaural-mss/data/binaural_musdb18/standard/test/'

In [ ]:
file_list = [f for f in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, f))]

In [ ]:
song = []
source = []
angle = []

for file in file_list:
    with open(os.path.join(DATA_DIR, file, 'metadata.json'), 'rb') as f:
        data = json.load(f)
        for k, v in data.items():
            if v > 90:
                v = v - 360
            song.append(file)
            source.append(k)
            angle.append(v)

In [ ]:
standard_angles = pd.DataFrame({'title': song, 'source': source, 'standard_angle': angle})

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6), tight_layout=True)

sns.histplot(x='standard_angle', hue='source', multiple='stack', data=standard_angles, binwidth=5, ax=ax)

ax.invert_xaxis()
ax.set(ylabel='Count', xlabel='$\Theta$')
fig.suptitle("Distribution of Angles in Binaural MUSDB18 (Standard Placement)")
plt.savefig('../data/plots/angles_standard.pdf', bbox_inches='tight')
plt.show()

**Random Placement**

In [ ]:
DATA_DIR = '/Users/richanamballa/Desktop/binaural-mss/data/binaural_musdb18/random/test/'

In [ ]:
file_list = [f for f in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, f))]

In [ ]:
song = []
source = []
angle = []

for file in file_list:
    with open(os.path.join(DATA_DIR, file, 'metadata.json'), 'rb') as f:
        data = json.load(f)
        for k, v in data.items():
            if v > 90:
                v = v - 360
            song.append(file)
            source.append(k)
            angle.append(v)

In [ ]:
random_angles = pd.DataFrame({'title': song, 'source': source, 'random_angle': angle})

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6), tight_layout=True)

sns.histplot(x='random_angle', hue='source', multiple='stack', data=random_angles, binwidth=10, ax=ax)

ax.invert_xaxis()
ax.set(ylabel='Count', xlabel='$\Theta$')
fig.suptitle("Distribution of Angles in Binaural MUSDB18 (Random Placement)")
plt.savefig('../data/plots/angles_random.pdf', bbox_inches='tight')
plt.show()

### Export Metadata

In [ ]:
merged_df = pd.merge(standard_angles, random_angles, how='inner', on=['title', 'source']).sort_values(by=['title', 'source'], ignore_index=True)

In [ ]:
merged_df.to_csv('../data/binaural_musdb18/metadata.csv')